In [1]:
## arxiv research

## tools creatuion
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
# used the inbuilt tools of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper= api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki,arxiv]

In [5]:
## custom tools [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embedding = OllamaEmbeddings(model="nomic-embed-text")
vector_db = FAISS.from_documents(documents,embedding)
retriever = vector_db.as_retriever()
retriever

/var/folders/bh/vz2kg8c14s97ywrs359cgrj00000gn/T/ipykernel_16941/2453116746.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1217e90f0>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,'langsmith-search',"Search any information about langsmith")

retriever_tool.name

'langsmith-search'

In [8]:
tools = [wiki,arxiv,retriever_tool]

In [9]:
from langchain_groq import ChatGroq
import os
import streamlit as st
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key,model_name = 'Llama3-8b-8192')

In [10]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "False"


In [11]:
### create prompt template

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/nikhilbansal/Documents/projects/GenAI_Chatbots/SearchEngine/venv/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [13]:
##agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools = tools,verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with Lan

{'input': 'Tell me about Langsmith',
 'output': 'It seems like Langsmith is a platform designed for building production-grade Large Language Model (LLM) applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, making it possible to ship quickly and with confidence.'}

In [15]:
agent_executor.invoke({"input":"What is machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result, it seems that the user's question about machine learning has been partially answered. However, it would be beneficial to provide a more concise and clear definition of machine learning.

Here's a response that builds upon the result from the tool:

Machine learning is a type of AI that enables computers to learn from data without being explicitly programmed. It involves developing algorithms that can analyze and make predictions on new, unseen data. In other words, machine learning allows computers to learn from experience and improve their performance over time.

If you'd like to know more about the applications or typ

{'input': 'What is machine learning?',
 'output': "Based on the result, it seems that the user's question about machine learning has been partially answered. However, it would be beneficial to provide a more concise and clear definition of machine learning.\n\nHere's a response that builds upon the result from the tool:\n\nMachine learning is a type of AI that enables computers to learn from data without being explicitly programmed. It involves developing algorithms that can analyze and make predictions on new, unseen data. In other words, machine learning allows computers to learn from experience and improve their performance over time.\n\nIf you'd like to know more about the applications or types of machine learning, feel free to ask!"}

In [16]:
agent_executor.invoke({"input":"What is this paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" by Vaswani et al. introduced the transformer architecture, which has since become widely used in natural language processing tasks.

> Finished chain.


{'input': 'What is this paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" by Vaswani et al. introduced the transformer architecture, which has since become widely used in natural language processing tasks.'}